In [ ]:
# 3、T1GARCH.py

from scipy import stats
import statsmodels.api as sm  # 统计相关的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arch  # 条件异方差模型相关的库

import seaborn as sns  # seaborn 画出的图更好看，且代码更简单
sns.set(color_codes=True)  # seaborn 设置背景

# 导入数据
data = pd.read_excel('test.xls')
data.set_index('date', inplace=True)  # 设定日期为索引
r2 = np.log(data['close']) - np.log(data['close'].shift(1))  # 计算对数收益率
r2 = r2.dropna()
r2.head()
r = r2

r2 = pd.DataFrame(r)
r2.columns = ['return']
r2.plot(figsize=(12, 4))

t = sm.tsa.stattools.adfuller(r2)
print("p-value:", t[1])

from statsmodels.graphics.tsaplots import plot_pacf as PACF

fig = PACF(r2, lags=20)
# plt.show()

from scipy import stats
import statsmodels.api as sm  # 统计相关的库
from statsmodels.tsa.ar_model import AutoReg

temp = np.array(r2)  # 载入收益率序列
model = AutoReg(temp, lags=[1, 6, 25, 26])
res = model.fit()
out = 'AIC: {0:0.3f}, HQIC: {1:0.3f}, BIC: {2:0.3f}'
print(out.format(res.aic, res.hqic, res.bic))
print(res.summary())

at = r2.values[26:] - res.fittedvalues
at2 = np.square(at)
m = 25  # 我们检验25个自相关系数
at2_flattened = np.ravel(at2)

# 计算自相关系数及 p-value
acf, q, p = sm.tsa.acf(at2_flattened, nlags=m, qstat=True)
out = np.c_[range(1, 26), acf[1:], q, p]
output = pd.DataFrame(out, columns=['lag', "AC", "Q", "P-value"])
output = output.set_index('lag')
at2 = at2.ravel()

fig = plt.figure(figsize=(20, 5))
fig = PACF(at2, lags=20)
print(sm.tsa.arma_order_select_ic(at2, max_ar=10, max_ma=0, ic='aic')['aic_min_order'])

am = arch.arch_model(r2.values, mean='AR', lags=20, vol='ARCH', p=10)
res = am.fit(options={"maxiter": 990})
res.summary()

In [ ]:
# LSTM 预测模型.py

import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arch
import seaborn as sns

data = pd.read_excel("test.xls")

print(data.head())
series = data.set_index(['Date'], drop=True)
plt.figure(figsize=(10, 6))

series['close'].plot()
# plt.show()

def difference(data_set, interval=1):
    diff = list()
    for i in range(interval, len(data_set)):
        value = data_set[i] - data_set[i - interval]
        diff.append(value)
    return pd.Series(diff)

# 这里的 series 是之前数据预处理后得到的 DataFrame 型数据
raw_value = series.values
diff_value = difference(raw_value, 1)

def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

supervised = timeseries_to_supervised(diff_value, 1)
supervised_value = supervised.values

# 这里的 series 是之前数据预处理后得到的 DataFrame 型数据
raw_value = series.values
diff_value = difference(raw_value, 1)
testNum = 6000
train, test = supervised_value[:-testNum], supervised_value[-testNum:]

In [ ]:
#机器学习.py
import pandas as pd
import numpy as np
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 准备数据
# 假设有一个包含收盘价的数据集，命名为 df，其中有时间序列数据和收盘价列 'Close'

# 计算收益率并创建 GARCH 模型
returns = df['Close'].pct_change().dropna() # 计算收益率
garch_model = arch_model(returns, vol='Garch', p=1, q=1) # 创建 GARCH (1,1) 模型
garch_fit = garch_model.fit() # 拟合 GARCH 模型

# 使用 GARCH 模型预测波动率
volatility_forecast = garch_fit.forecast(horizon=1).variance[-1:].values[0] # 预测未 来波动率

# 构建 LSTM 模型
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=256, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=128))
    model.add(Dense(1))
    return model

# 准备数据集
def prepare_data(data, look_back=10):
    X, y = [], []
    for i in range(len(data)-look_back):
        X.append(data[i:i+look_back])
        y.append(data[i+look_back])
    return np.array(X), np.array(y)

# 归一化数据
scaler = MinMaxScaler(feature_range=(0, 1))
returns_scaled = scaler.fit_transform(returns.values.reshape(-1, 1))

# 准备训练数据
look_back = 10
X_train, y_train = prepare_data(returns_scaled, look_back)

# 构建并训练 LSTM 模型
lstm_model = create_lstm_model((look_back, 1))
lstm_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32)

# 进行波动率预测
X_test = returns_scaled[-look_back:].reshape(1, look_back, 1)
volatility_lstm = lstm_model.predict(X_test)[0][0]

# 组合 GARCH 模型和 LSTM 模型的预测结果
combined_forecast = volatility_forecast * volatility_lstm

# 输出波动率预测结果
print(f"组合预测的波动率: {combined_forecast}")

# 反归一化得到实际值
actual_volatility = scaler.inverse_transform(combined_forecast.reshape(-1, 1))

# 可视化预测结果
import matplotlib.pyplot as plt

plt.plot(df['Date'], df['Close'], label='Actual Close Price')
plt.plot(df['Date'].iloc[-1], actual_volatility, marker='o', markersize=10, label='Volatility Forecast')
plt.xlabel('日期')
plt.ylabel('价格/波动率')
plt.legend()
plt.show()



IndentationError: unexpected indent (4260449461.py, line 18)

In [ ]:
# 机器学习 LSTM.py
import pandas as pd
import numpy as np
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 准备数据
# 假设有一个包含收盘价的数据集，命名为 df，其中有时间序列数据和收盘价列 'Close'

# 计算收益率并创建 GARCH 模型
returns = df['Close'].pct_change().dropna()  # 计算收益率
garch_model = arch_model(returns, vol='Garch', p=1, q=1)  # 创建 GARCH (1,1) 模型
garch_fit = garch_model.fit()  # 拟合 GARCH 模型

# 使用 GARCH 模型预测波动率
volatility_forecast = garch_fit.forecast(horizon=1).variance[-1:].values[0]  # 预测未来波动率

# 构建 LSTM 模型
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=256, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=128))
    model.add(Dense(1))
    return model

# 准备数据集
def prepare_data(data, look_back=10):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

# 归一化数据
scaler = MinMaxScaler(feature_range=(0, 1))
returns_scaled = scaler.fit_transform(returns.values.reshape(-1, 1))

# 准备训练数据
look_back = 10
X_train, y_train = prepare_data(returns_scaled, look_back)

# 构建并训练 LSTM 模型
lstm_model = create_lstm_model((look_back, 1))
lstm_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32)

# 进行波动率预测
X_test = returns_scaled[-look_back:].reshape(1, look_back, 1)
volatility_lstm = lstm_model.predict(X_test)[0][0]

# 组合 GARCH 模型和 LSTM 模型的预测结果
combined_forecast = volatility_forecast * volatility_lstm

# 输出波动率预测结果
print(f"组合预测的波动率: {combined_forecast}")

# 反归一化得到实际值
actual_volatility = scaler.inverse_transform(combined_forecast.reshape(-1, 1))

# 可视化预测结果
import matplotlib.pyplot as plt

plt.plot(df['Date'], df['Close'], label='Actual Close Price')
plt.plot(df['Date'].iloc[-1], actual_volatility, marker='o', markersize=10, label='Volatility Forecast')
plt.xlabel('日期')
plt.ylabel('价格/波动率')
plt.legend()
plt.show()

In [ ]:
# 调用数据接口.py
from iFinDPy import *
from datetime import datetime
import pandas as pd
import time as _time
import json
from threading import Thread, Lock, Semaphore
import requests

sem = Semaphore(5)  # 此变量用于控制最大并发数
dllock = Lock()  # 此变量用来控制实时行情推送中落数据到本地的锁

# 登录函数
def thslogindemo():
    # 输入用户的帐号和密码
    thsLogin = THS_iFinDLogin("gbafm728", "2a8bb2")
    print(thsLogin)
    if thsLogin != 0:
        print('登录失败')
    else:
        print('登录成功')

def datepool_basicdata_demo():
    # 通过数据池的板块成分函数和基础数据函数，提取沪深300 的全部股票在2020-11-16 日的日不复权收盘价
    data_hs300 = THS_DP('block', '2020-11-16;001005290', 'date:Y,thscode:Y,security_name:Y')
    if data_hs300.errorcode != 0:
        print('error:{}'.format(data_hs300.errmsg))
    else:
        seccode_hs300_list = data_hs300.data['THSCODE'].tolist()
        data_result = THS_BD(seccode_hs300_list, 'ths_close_price_stock', '2020-11-16,100')
        if data_result.errorcode != 0:
            print('error:{}'.format(data_result.errmsg))
        else:
            data_df = data_result.data
            print(data_df)

def datapool_realtime_demo():
    # 通过数据池的板块成分函数和实时行情函数，提取上证50 的全部股票的最新价数据，并将其导出为csv 文件
    today_str = datetime.today().strftime('%Y-%m-%d')
    print('today:{}'.format(today_str))
    data_sz50 = THS_DP('block', '{};001005260'.format(today_str), 'date:Y,thscode:Y,security_name:Y')
    if data_sz50.errorcode != 0:
        print('error:{}'.format(data_sz50.errmsg))
    else:
        seccode_sz50_list = data_sz50.data['THSCODE'].tolist()
        data_result = THS_RQ(seccode_sz50_list, 'latest')
        if data_result.errorcode != 0:
            print('error:{}'.format(data_result.errmsg))
        else:
            data_df = data_result.data
            print(data_df)
            data_df.to_csv('realtimedata_{}.csv'.format(today_str))

def iwencai_demo():
    # 演示如何通过不消耗流量的自然语言语句调用常用数据
    print('输出资金流向数据')
    data_wencai_zjlx = THS_WC('主力资金流向', 'stock')
    if data_wencai_zjlx.errorcode != 0:
        print('error:{}'.format(data_wencai_zjlx.errmsg))
    else:
        print(data_wencai_zjlx.data)

    print('输出股性评分数据')
    data_wencai_xny = THS_WC('股性评分', 'stock')
    if data_wencai_xny.errorcode != 0:
        print('error:{}'.format(data_wencai_xny.errmsg))
    else:
        print(data_wencai_xny.data)

def dlwork(tick_data):
    # 本函数为实时行情订阅新启线程的任务函数
    dllock.acquire()
    with open('dlwork.txt', 'a') as f:
        for stock_data in tick_data['tables']:
            if 'time' in stock_data:
                timestr = _time.strftime('%Y-%m-%d %H:%M:%S', _time.localtime(stock_data['time'][0]))
                print(timestr)
                f.write(timestr + str(stock_data) + '\n')
            else:
                pass
    dllock.release()

def work(codestr, lock, indilist):
    sem.acquire()
    stockdata = THS_HF(codestr, ';'.join(indilist), '', '2020-08-11 09:15:00', '2020-08-11 15:30:00', 'format:json')
    if stockdata.errorcode != 0:
        print('error:{}'.format(stockdata.errmsg))
        sem.release()
    else:
        print(stockdata.data)
        lock.acquire()
        with open('test1.txt', 'a') as f:
            f.write(str(stockdata.data) + '\n')
        lock.release()
        sem.release()

def multiThread_demo():
    # 本函数为通过高频序列函数，演示如何使用多线程加速数据提取的示例，本例中通过将所有A 股分100 组，最大线程数量sem 进行提取
    # 用户可以根据自身场景进行修改
    today_str = datetime.today().strftime('%Y-%m-%d')
    print('today:{}'.format(today_str))
    data_alla = THS_DP('block', '{};001005010'.format(today_str), 'date:Y,thscode:Y,security_name:Y')
    if data_alla.errorcode != 0:
        print('error:{}'.format(data_alla.errmsg))
    else:
        stock_list = data_alla.data['THSCODE'].tolist()
    indi_list = ['close', 'high', 'low', 'volume']
    lock = Lock()

    btime = datetime.now()
    l = []
    for eachlist in [stock_list[i:i + int(len(stock_list) / 10)] for i in range(0, len(stock_list), int(len(stock_list) / 10))]:
        nowstr = ','.join(eachlist)
        p = Thread(target=work, args=(nowstr, lock, indi_list))
        l.append(p)

    for p in l:
        p.start()
    for p in l:
        p.join()
    etime = datetime.now()
    print(etime - btime)

pd.options.display.width = 320
pd.options.display.max_columns = None

def reportDownload():
    df = THS_ReportQuery('300033.SZ', 'beginrDate:2021-08-01;endrDate:2021-08-31;reportType:901', 'reportDate:Y,thscode:Y,secName:Y,ctime:Y,reportTitle:Y,pdfURL:Y,seq:Y').data
    print(df)
    for i in range(len(df)):
        pdfName = df.iloc[i, 4] + str(df.iloc[i, 6]) + '.pdf'
        pdfURL = df.iloc[i, 5]
        r = requests.get(pdfURL)
        with open(pdfName, 'wb+') as f:
            f.write(r.content)

def main():
    # 本脚本为数据接口通用场景的实例，可以通过取消注释下列示例函数来观察效果
    # 登录函数
    thslogindemo()
    # 通过数据池的板块成分函数和基础数据函数，提取沪深300 的全部股票在2020-11-16 日的日不复权收盘价
    # datepool_basicdata_demo()
    # 通过数据池的板块成分函数和实时行情函数，提取上证50 的全部股票的最新价数据，并将其导出为csv 文件
    # datapool_realtime_demo()
    # 演示如何通过不消耗流量的自然语言语句调用常用数据
    # iwencai_demo()

if __name__ == '__main__':
    main()

In [ ]:
# 获取实时 IOPV.py

import requests


def get_etf_iopv(symbol):
    url = "http://webstock.quote.hermes.hexun.com/513130.shtml"

    headers = {
        "Referer": "http://quote.hexun.com/hqetf/default.html"
    }

    response = requests.get(url, headers=headers)
    data = response.text
    start_index = data.find("ID_hq_etf_curPrice='") + len("ID_hq_etf_curPrice='")
    end_index = data.find("'", start_index)
    iopv = float(data[start_index:end_index])

    return iopv


iopv = get_etf_iopv('513130')
print(iopv)

In [ ]:
# 任务一回测.py

import numpy as np
import pandas as pd


def init(context):
    set_commission(PerShare(type='future', cost=0.00005))
    set_commission(PerShare(type='stock', cost=0.001))

    set_subportfolios([
        {'cash': 500000, 'type': 'stock'},
        {'cash': 1000000, 'type': 'future'}
    ])

    # 设置时间周期
    context.m = 20
    # 设置要交易的股票
    context.stocks = ['513130.SH']


# 设置买卖条件，每个交易频率（日/分钟/tick）调用一次
def handle_bar(context, bar_dict):
    num = context.m
    for stk in context.stocks:
        # 获取股票历史收盘价数据
        close = history(stk, ['close'], 3 * num, '1d')
        c = close['close'].values
        EGARCH20 = EGARCH(close.values, num)

        if c[-1] > EGARCH20[-1] and c[-2] < EGARCH20[-2] and stk not in list(context.portfolio.stock_account.positions.keys()):
            order_target_percent(stk, 1)

        if c[-1] < EGARCH20[-1] and c[-2] > EGARCH20[-2] and stk in list(context.portfolio.stock_account.positions.keys()):
            order_target(stk, 0)


def getWindowMatrix(inputArray, t, m):
    temp = []
    n = t - m + 1
    for i in range(n):
        tmp = []
        for j in range(m):
            tmp.append(inputArray[i + j][0])
        temp.append(tmp)
    WindowMatrix = np.array(temp)
    return WindowMatrix


def SVDreduce(WindowMatrix):
    u, s, v = np.linalg.svd(WindowMatrix)
    m1, n1 = u.shape
    m2, n2 = v.shape
    index = s.argmax()
    u1 = u[:, index]
    v1 = v[index]
    u1 = u1.reshape((m1, 1))
    v1 = v1.reshape((1, n2))
    value = s.max()
    newMatrix = value * (np.dot(u1, v1))
    return newMatrix


def recreateArray(newMatrix, t, m):
    ret = []
    n = t - m + 1
    for p in range(1, t + 1):
        if p < m:
            alpha = p
        elif p > t - m + 1:
            alpha = t - p + 1
        else:
            alpha = m
        sigma = 0
        for j in range(1, m + 1):
            i = p - j + 1
            if 0 < i <= n:
                sigma += newMatrix[i - 1][j - 1]
        ret.append(sigma / alpha)
    return ret


def EGARCH(inputArray, m):
    t = 2 * m
    WindowMatrix = getWindowMatrix(inputArray, t, m)
    newMatrix = SVDreduce(WindowMatrix)
    newArray = recreateArray(newMatrix, t, m)
    return newArray

In [ ]:
# 任务二回测.py

import numpy as np
import pandas as pd
import statsmodels.api as sm


def init(context):
    set_commission(PerShare(type='future', cost=0.00005))
    set_commission(PerShare(type='stock', cost=0.001))

    set_subportfolios([
        {'cash': 500000, 'type': 'stock'},
        {'cash': 1000000, 'type': 'future'}
    ])
    set_params(context)
    set_variables(context)
    set_backtest()


def set_params(context):
    pass


def handle_bar(context, bar_dict):
    if context.X_length > 2:
        Slope = get_signal(context, bar_dict)
        trade_signal(context, Slope)
        trade_operate(context)


def get_signal(context, bar_dict):
    value = history(context.stock, ['close', 'open'], context.X_length, '1d', True)
    X = np.array(range(context.X_length))
    X2 = X ** 2
    c = np.ones(len(X))
    Y = (value.close.values + value.open.values) / 2
    dic1 = {'X2': X2, 'X': X, 'c': c, 'y': Y}
    df = pd.DataFrame(dic1)
    x_train = df[['X', 'X2', 'c']]
    y_train = df[['y']]
    model = sm.OLS(y_train, x_train)
    results = model.fit()
    Slope = results.params[0] + 2 * results.params[1] * df.X.values[-1]
    log.info(Slope)
    return Slope


def trade_signal(context, Slope):
    for i in context.Countdown.keys():
        context.Countdown[i] -= 1
        if context.Countdown[i] == 0 and context.direct * Slope < 0:
            log.info("方向更改", context.direct, Slope)
            context.direct = Slope
            context.is_change = True
            return
    if context.direct == 0:
        context.direct = Slope
        if Slope > 0:
            order_target_percent(context.stock, 1)
    else:
        if context.direct * Slope < 0:
            context.Countdown[context.X_length] = context.N


def trade_operate(context):
    if context.is_change:
        if context.direct > 0:
            order_target_percent(context.stock, 1)
        if context.direct < 0:
            order_target_percent(context.stock, 0)
        context.X_length = 1
        context.is_change = False
        context.Countdown = {}


def after_trading(context):
    context.X_length += 1

In [ ]:
# 任务三回测.py

import pandas as pd
import arch as arch
import datetime
import numpy as np
from datetime import datetime


def init(context):
    set_commission(PerShare(type='future', cost=0.00005))
    set_commission(PerShare(type='stock', cost=0.001))

    set_subportfolios([
        {'cash': 500000, 'type': 'stock'},
        {'cash': 1000000, 'type': 'future'}
    ])

    context.security = '513130.SH'


def handle_bar(context, bar_dict):
    stk = context.security
    # 获取回测当天日期时间
    # time = get_datetime()
    data = history(stk, ['open', 'high', 'low'], 80, '1d')
    # log.info(data)
    # today = int(time.strftime("%Y%m%d"))
    # value = get_price('513130.SH', None, 'today', '1d', ['open', 'high', 'low'], True, None, 250, is_panel=1)
    # log.info(value)

    log_return = pd.DataFrame({
        'open': np.log(data['open']).diff(),
        'high': np.log(data['high']).diff(),
        'low': np.log(data['low']).diff()
    })
    log_return.dropna(how='any', inplace=True)
    # log.info(log_return)

    # egarch_model = arch.arch_model(log_return['open'].values, vol='EGARCH', p=1, o=0, q=1)
    # egarch_res = egarch_model.fit()
    # forecast = egarch_res.forecast(horizon=5, method='simulation')
    # forecast_data = pd.DataFrame({
    #     'open': np.exp(forecast.mean['h.1']),
    #     'high': np.exp(forecast.mean['h.2']),
    #     'low': np.exp(forecast.mean['h.3'])
    # })

    # # dif = (forecast_data['high'] + forecast_data['low']) / forecast_data['open'] - 2
    # dif = (data['high'] + data['low']) / data['open'] - 2
    dif = (log_return['high'] + log_return['low']) / log_return['open'] - 2.23
    dif_ma = pd.rolling_mean(dif, 60)
    # log.info(dif_ma.values[-1])

    if dif_ma.values[-1] > 0:
        order_target_percent(stk, 1)
    if dif_ma.values[-1] < 0 and context.portfolio.stock_account.market_value > 0:
        order_target(stk, 0)